# DecisionTree

In [1]:
import numpy as np

## ID3和C4.5算法实现：

In [2]:
class TreeNode():
    """
    树结点
    """
    def __init__(self, node=None, edge=None, leaf_node=None, child_node=None ):
        """
        node: 当前结点，储存的是当前划分的特征的索引
        
        edge： 边，储存的是划分特征的不同取值
        
        leaf_node: 叶节点，用于储存类别标签
        
        child_node: 子节点，储存下一层树的结点，字典形式{edge：nextnode}
        """
        self.node = node
        self.edge = edge
        self.leaf_node = leaf_node
        self.child_node = child_node
    

In [3]:
class DecisionTreeClassifier():
    def __init__(self, entropy='gain', epsilon=0.01, max_depth=None):
        """
        参数：
        entropy = 'gain' or 'gain_ratio'：表示使用信息增益或信息增益比作为特征选择准则
        
        epsilon：阈值，小于这个阈值则停止划分特征
        
        max_depth：若树超过这个深度则停止生长
        
        root: 储存生成的树结构
        """
        self.root = None
        self.entropy = entropy
        self.epsilon = epsilon
        self.max_depth = max_depth

    
    def calc_entropy(self, y):
        """
        计算熵
        
        y：n×1维向量，为数据集的标签向量
        """
        entropy = 0
        labels, counts = np.unique(y,return_counts=True)#返回一个2个元素的元组，第一个元素为类别数组，第二个为计数数组
        p = counts/(np.sum(counts))
        entropy = np.sum(-p * np.log2(p))
        return entropy
    
    def calc_cond_entropy(self, x, y):
        """
        计算条件熵
        
        x: n×1维向量，代表某一个特征上的数据
        y: n×1维向量,对应的类别向量
        """
        cond_entropy=0
        x_val, val_counts = np.unique(x, return_counts=True)#对特征x的取值进行计数
        for val, counts in zip(x_val,val_counts):
            y_sub = y[x==val]
            entropy = self.calc_entropy(y_sub)
            p = counts/len(x)
            cond_entropy += p*entropy
        return cond_entropy
    
    def calc_gain(self, x, y):
        """
        计算信息增益（特征同上）
        """
        print(self.calc_entropy(y) - self.calc_cond_entropy(x,y))
        return self.calc_entropy(y) - self.calc_cond_entropy(x,y)#熵减去条件熵
    
    def calc_gain_ratio(self, x, y):
        """
        计算信息增益比（特征同上）
        """
        return self.calc_gain(x,y) / self.calc_entropy(x)#信息增益/特征x的熵
    
    def vote(self, y):
        """
        投票法决定最后叶节点的值
        """
        labels, counts = np.unique(y,return_counts=True)#计数
        return labels[np.argmax(counts)]#返回出现次数最多的标签作为叶节点的值
        
    def build_tree(self, X, y, feature_set,depth=0):
        """
        递归生成一棵树：
        
        X：二维数组，数据集
        y：一维向量，标签
        feature_set: 子树可供选择的特征集
        depth： 树的当前深度
        """
        # 当子树中的所有数据都属于同一类时，将该子树设为叶节点
        if np.unique(y).shape[0] == 1:
            return TreeNode(leaf_node=self.vote(y))
        
        
        #当子树特征集为空时，将该子树设为叶节点           
        if not feature_set:                           #空集合默认为False
            return TreeNode(leaf_node=self.vote(y))
        
        #当子树的深度达到最大深度时，将该子树设为叶节点
        if depth == self.max_depth:
            return TreeNode(leaf_node=self.vote(y))
        
        #选择信息增益或信息增益比最大的特征
        max_info = 0   #记录最大信息增益或信息增益比
        node_feature = 0       #记录对应的特征索引
        for i in feature_set:
            if self.entropy == 'gain':                 #计算特征的信息增益或信息增益比
                info = self.calc_gain(X[:,i],y)
            elif self.entropy == 'gain_ratio':
                info = self.calc_gain_ratio(X[:,i],y)
                
            if info > max_info :                       #更新最大信息增益（比）的值以及对应的特征索引，此时特征索引应该排除之前选过的
                max_info = info
                node_feature = i
                
        #若最大的信息增益（比）小于阈值，则将该子树设为叶节点
        if max_info <= self.epsilon:
            return TreeNode(leaf_node=self.vote(y))
        
        #以最大信息增益（比）的特征取值划分子树
        child_tree={}
        feature_values = np.unique(X[:,node_feature])            # 特征取值
        for val in feature_values:
            child_X = X[X[:,node_feature]==val]                  #按特征某个取值划分出子数据和子标签
            child_y = y[X[:,node_feature]==val]
            #child_X = np.delete(child_X, node_feature, 1)        #注：这里一开始犯了一个错误，删除特征不可以这样做，会导致删除特征后面
                                                                  #的特征位置-1，索引的时候就会出现错误，因此修改为添加了特征集
            new_feature = feature_set.copy()
            new_feature.remove(node_feature)
            new_depth = depth + 1
            child_tree[val] = self.build_tree(child_X,child_y,new_feature,new_depth)    #递归生成决策树,
                                                                                        #子树不需要这个特征来进行划分了，需要删掉
        return TreeNode(node=node_feature, edge=feature_values, child_node=child_tree)
    
    def search_tree(self, x, tree):
        """
        遍历搜索树：
        
        x：一维数组，测试数据
        tree: 给定的树类，用于搜索到叶节点来给测试数据分类
        """
        #先判断这个结点是否为叶结点，搜索到叶结点时，返回其储存的类别
        if tree.leaf_node != None:
            return tree.leaf_node 
        
        #搜索到内部节点时，递归搜索子树
        for val, next_tree in tree.child_node.items():    
            if x[tree.node] == val:                       
                return self.search_tree(x, next_tree)
            
            
    def fit(self, X,y):
        """
        训练模型就是生成一棵决策树的过程

        X：二维数组，训练数据集
        y：一维数组，标签
        """

        self.root = self.build_tree(X,y,set(range(X.shape[1])))             #将生成的树保存在root变量中


    def predict(self,x):
        """
        给测试数据分类就是搜索树的过程

        x：一维数组，测试数据
        """
        return self.search_tree(x,self.root)


In [4]:
import pandas as pd

In [5]:

datasets = [['青年', '否', '否', '一般', '否'],
           ['青年', '否', '否', '好', '否'],
           ['青年', '是', '否', '好', '是'],
           ['青年', '是', '是', '一般', '是'],
           ['青年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '一般', '否'],
           ['中年', '否', '否', '好', '否'],
           ['中年', '是', '是', '好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['中年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '非常好', '是'],
           ['老年', '否', '是', '好', '是'],
           ['老年', '是', '否', '好', '是'],
           ['老年', '是', '否', '非常好', '是'],
           ['老年', '否', '否', '一般', '否'],
           ]
labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
train_data = pd.DataFrame(datasets, columns=labels)
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [6]:
train_data = train_data[['有自己的房子','信贷情况','年龄','有工作','类别']]

In [7]:
X = train_data.values[:,0:-1]
y = train_data.values[:,-1]


In [8]:
clf = DecisionTreeClassifier()
clf.fit(X,y)

0.4199730940219749
0.36298956253708536
0.08300749985576883
0.32365019815155627
0.47385138961004514
0.2516291673878229
0.9182958340544896


In [9]:
clf.root.child_node['否'].node

3

In [10]:
clf.root.node

0

In [251]:
a=set(range(X.shape[1]))

In [13]:
a = np.arange(9).reshape([3,3])

In [14]:
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [16]:
np.delete(a, 2, 1).shape

(3, 2)

In [188]:
dt = DecisionTreeScratch(feature_name=labels)
dt.fit(X,y)

In [190]:
dt._root._child['否']._feature_idx

2